### Purposes
1. Merge the 2017 ACS 5-year estimate household income distribution data with the 2019 TIGER/Line shapefile of Census Tracts in Brooklyn and Manhattan.
2. Read in the Neighborhood Tabulation Area (NTA) data for use as a scale of geospatial analysis.
3. Aggregate the 2017 5-year estimate household income distribution data at the NTA scale
4. Read in the life expectancy data, aggregate, and merge at the NTA scale.
5. Merge the two datasets above with the georeferenced Yelp data

In [44]:
# Setting up modules
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

# Data path 
path_census_tracts = './data/ACS/'
path_yelp = './data/Yelp/'

In [45]:
# Import csvs
BK_hh_income_dis_pct = pd.read_csv(path_census_tracts + 'BK_hh_income_dis_pct.csv')
BK_hh_income_dis_count = pd.read_csv(path_census_tracts + "BK_hh_income_dis_count.csv")
# BK_hh_income_dis_pct = pd.DataFrame(BK_hh_income_dis_pct)
NTA_census_tract_equivalency_table = pd.read_csv(path_census_tracts + "CensusTracts/nyc2010census_tabulation_equiv.csv")
MN_hh_income_dis_count_NTA = pd.read_csv(path_census_tracts + "CensusTracts/ManhattanNTAIncomeDistribution.csv")
NYC_LifeExpectancy = pd.read_csv(path_census_tracts + "CensusTracts/NYC_LifeExpectancy.csv")

# Import shps
NY_state_census_tracts = gpd.read_file(path_census_tracts + "CensusTracts/tl_2019_36_tract.shp")
MN_Yelp = gpd.read_file(path_yelp + "MN/MN_Yelp.shp")
BK_Yelp = gpd.read_file(path_yelp + "BK/BK_Yelp.shp")
NYC_neighborhood_tabn_areas = gpd.read_file(path_census_tracts + "CensusTracts/nynta_reproj.shp")

In [29]:
MN_hh_income_dis_count_NTA.head()

,NTA_CODE,0-25k,25-50k,50-75k,75-100k,100-125k,125-150k,> 150k
0,MN01,0.32,0.21,0.15,0.14,0.07,0.04,0.07
1,MN03,0.37,0.22,0.15,0.09,0.06,0.04,0.07
2,MN04,0.28,0.24,0.18,0.11,0.06,0.05,0.09
3,MN06,0.41,0.25,0.11,0.08,0.06,0.03,0.05
4,MN09,0.30,0.14,0.13,0.08,0.07,0.05,0.23


In [30]:
BK_hh_income_dis_count.head()

,ID,Geography,Households,0-25k,25k-50k,50k-75k,75k-100k,100k-125k,125k-150k,> 150k
0,36047000100,"Census Tract 1, Kings County, New York",2184,464,217,190,402,145,153,613
1,36047000200,"Census Tract 2, Kings County, New York",377,99,115,78,26,47,0,12
2,36047000301,"Census Tract 3.01, Kings County, New York",1865,81,382,16,200,232,142,812
3,36047000501,"Census Tract 5.01, Kings County, New York",1772,333,74,167,99,169,208,722
4,36047000502,"Census Tract 5.02, Kings County, New York",1560,206,180,116,167,155,113,623


In [31]:
# Filter for NYC (Manhattan and Brooklyn)

BK = NY_state_census_tracts[NY_state_census_tracts["COUNTYFP"] == '047']
MN = NY_state_census_tracts[NY_state_census_tracts["COUNTYFP"] == '061']
BK_neighborhood_tabn_areas = NYC_neighborhood_tabn_areas[NYC_neighborhood_tabn_areas["BoroName"] == "Brooklyn"]
MN_neighborhood_tabn_areas = NYC_neighborhood_tabn_areas[NYC_neighborhood_tabn_areas["BoroName"] == "Manhattan"]

# Cutting out all of the crap
MN = MN[["GEOID", "geometry"]]
BK = BK[["GEOID", "geometry"]]
BK_neighborhood_tabn_areas = BK_neighborhood_tabn_areas[["NTACode", "NTAName", "geometry"]]
MN_neighborhood_tabn_areas = MN_neighborhood_tabn_areas[["NTACode", "NTAName", "geometry"]]

# Converting type of column
# int wont work because it is too long
BK.GEOID = BK.GEOID.astype(float)
# Merging
BK = BK.merge(BK_hh_income_dis_count, left_on = 'GEOID', right_on = 'ID')

In [32]:
# Reprojecting the BK_NTA file
#BK_neighborhood_tabn_areas.crs = {'init': 'epsg:4269'}
#MN_neighborhood_tabn_areas.crs = {'init': 'epsg:4269'}

# Are the CRSes identical?
print(BK.crs == BK_Yelp.crs)
print(BK.crs == BK_neighborhood_tabn_areas.crs)

True
True


At this point, we have a slight issue. The ACS data is collected at the Census tract level. We know that NTAs' borders line up perfectly with that of the census tracts. However, there is no column within the NTA shapefile that tells us which tracts each NTA is made up of. Hence, we need to perform a spatial join with the Brooklyn census tract file to create a column that identifies which NTA each tract belongs to so as to facilitate grouping and summary statistics. 

In [33]:
# Spatial join for Brooklyn ACS Census Tract with NTA
##BK_ct_NTA_joined = gpd.sjoin(BK, BK_neighborhood_tabn_areas, how = "inner", op = "intersects")
##BK_ct_NTA_joined = BK_ct_NTA_joined.drop(columns = ["ID","Geography", "index_right"])

In [34]:
##print("Number of census tracts in Brooklyn: " + str(BK.shape[0]))
##print("Number of NTAs in Brooklyn: " + str(BK_neighborhood_tabn_areas.shape[0]))
# Select all duplicate rows based on one column
##duplicate_rows_bk_df = BK_ct_NTA_joined[BK_ct_NTA_joined.duplicated(subset = ['GEOID'])]
##print("Number of duplicate rows based on GEOID post-spatial join: " +  str(duplicate_rows_bk_df.shape[0]))

Ideally, and more simply, I would do a spatial join as shown above between the NTA and Census Tract shapefiles to determine the NTA each census tract falls into. However, as the print-out above shows, there are multiple duplication of rows post spatial join. Why? The `op` argument in `gpd.sjoin` accepts one of three values: `"intersects", "contains", "within"`. The following [site](https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates) by the developers of the `shapely` module describes the different types of spatial joins.   

All three are unsuitable as the borders of each NTA and Census Tract do not line up **perfectly**. In other words there are some overlaps here and there which throw off the entire spatial join. Even if the borders line up perfectly, contiguity is an issue. In an `intersect`, it does not matter which side of the line an object is on as long as it touches the line unless a direction argument is available and specified, neither of which are available in this case. That is to say, tract i will be counted as part of both NTAs j and k as long as the border of tract i lines up along the border between NTAs j and k. I imagine common vertices count too. 

Hence, I had to use an equivalency table published by [NYC Planning](https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-nynta.page) to map each census tract to the NTA code. This is done using the GEOID column which is common to both tables. The GEOID column in the equivalency table is a new column I created by concatenating State FIPS (36) with County FIPS and Tract ID 
in Excel before saving and exporting as a CSV. Some table structure cleaning was also performed in Excel prior.   

In [35]:
# Merging
BK_NTA = BK.merge(NTA_census_tract_equivalency_table, left_on = 'ID', right_on = 'GEOID')

# Reducing the width
BK_NTA_reduced = BK_NTA.drop(["GEOID_x", "Geography", "State FIPS", "County FIPS", "2010 Census Tract", "GEOID_y", "Borough", "ID"], axis = 1)

In [36]:
print("Number of Census Tracts in newly merged file == Number of Census Tracts in Brooklyn? " + str(BK_NTA_reduced.shape[0] == BK.shape[0]))

Number of Census Tracts in newly merged file == Number of Census Tracts in Brooklyn? True


Okay so I have established that the merge ran smoothly with no duplications. The next thing to do is to run a summary function over the household income distribution columns that groups tracts by NTAs, summarize, and convert to percentages while keeping its geometry.

In [37]:
col_names = BK_NTA_reduced.columns
#print(col_names)
#print(BK_neighborhood_tabn_areas.head())

In [38]:
MN_neighborhood_tabn_areas.head()

,NTACode,NTAName,geometry
4,MN06,Manhattanville,"POLYGON ((-73.94607828608069 40.8212632160616,..."
49,MN15,Clinton,POLYGON ((-73.99383108070876 40.77293178785057...
82,MN27,Chinatown,"POLYGON ((-73.9838238729819 40.72147287206688,..."
84,MN25,Battery Park City-Lower Manhattan,(POLYGON ((-74.00078295209646 40.6942865161671...
86,MN14,Lincoln Square,POLYGON ((-73.97500111964057 40.77752777991669...


In [39]:
# Group, rowsum, col div by only the income columns, retaining the absolute number of households per NTA
BK_NTA_grouped = BK_NTA_reduced.groupby(by=col_names[10]).sum().iloc[:, 0:8]
BK_NTA_grouped.iloc[:, 1:] = BK_NTA_grouped.iloc[:, 1:].div(BK_NTA_grouped.iloc[:, 1:].sum(axis=1), axis=0)

# Merging with the BK NTA shp to get geometry that was lost with the merging, grouping and summary functions
BK_NTA_grouped_pct = BK_neighborhood_tabn_areas.merge(BK_NTA_grouped, left_on = 'NTAName', right_index = True)
MN_NTA_grouped_pct = MN_neighborhood_tabn_areas.merge(MN_hh_income_dis_count_NTA, left_on = "NTACode", right_on = "NTA_CODE" )

In [40]:
print("Number of NTAs in Brookyln in the merged file: " + str(BK_NTA_grouped_pct.shape[0]))
print("If there are 51, that means that the merge ran successfully.")

Number of NTAs in Brookyln in the merged file: 51
If there are 51, that means that the merge ran successfully.


In [41]:
MN_NTA_grouped_pct.head()

,NTACode,NTAName,geometry,NTA_CODE,0-25k,25-50k,50-75k,75-100k,100-125k,125-150k,> 150k
0,MN06,Manhattanville,"POLYGON ((-73.94607828608069 40.8212632160616,...",MN06,0.41,0.25,0.11,0.08,0.06,0.03,0.05
1,MN15,Clinton,POLYGON ((-73.99383108070876 40.77293178785057...,MN15,0.19,0.15,0.10,0.11,0.10,0.07,0.29
2,MN27,Chinatown,"POLYGON ((-73.9838238729819 40.72147287206688,...",MN27,0.38,0.19,0.13,0.10,0.05,0.04,0.12
3,MN25,Battery Park City-Lower Manhattan,(POLYGON ((-74.00078295209646 40.6942865161671...,MN25,0.11,0.06,0.07,0.08,0.11,0.08,0.50
4,MN14,Lincoln Square,POLYGON ((-73.97500111964057 40.77752777991669...,MN14,0.14,0.08,0.09,0.10,0.08,0.07,0.44


At this point, the ACS data have been successfully aggregated at the NTA scale. This means that we can now run spatial joins with the Yelp data to determine the NTA each restaurants falls into. 

In [42]:
# Spatial join for Yelp with Neighborhood Tabulation Area 
BK_NTA_Yelp_joined = gpd.sjoin(BK_Yelp, BK_NTA_grouped_pct, how = 'inner', op = 'within')
MN_NTA_Yelp_joined = gpd.sjoin(MN_Yelp, MN_NTA_grouped_pct, how = "inner", op = "within")

MN_NTA_Yelp_joined.head(2)

,id,alias,name,is_closed,review_cou,rating,price,categories,latitude,longitude,...,NTACode,NTAName,NTA_CODE,0-25k,25-50k,50-75k,75-100k,100-125k,125-150k,> 150k
0,ksksxd8J2SIs97ccPFV75A,brown-sugar-restaurant-new-york,Brown Sugar Restaurant,0,204,3.5,$$,"cuban,seafood,steak",40.869926,-73.915466,...,MN01,Marble Hill-Inwood,MN01,0.32,0.21,0.15,0.14,0.07,0.04,0.07
1,IHGm6huN_Z48KdorBjztSQ,guacamole-taqueria-new-york,Guacamole Taqueria,0,122,3.5,$$,mexican,40.869520,-73.916740,...,MN01,Marble Hill-Inwood,MN01,0.32,0.21,0.15,0.14,0.07,0.04,0.07


In [43]:
# Saving to file as .shp
## Impt to set encoding = 'utf-8' or it will throw a codec error
#MN_Yelp_NTA_joined.to_file(path_yelp + 'MN/MN_Yelp_CensusTract_NTA.shp', driver='ESRI Shapefile', encoding = 'utf-8')
BK_NTA_Yelp_joined.to_file(path_yelp + 'BK/BK_Yelp_CensusTract_NTA.shp', driver='ESRI Shapefile', encoding = 'utf-8')
MN_NTA_Yelp_joined.to_file(path_yelp + 'MN/MN_Yelp_CensusTract_NTA.shp', driver='ESRI Shapefile', encoding = 'utf-8')